In [1]:
%matplotlib inline

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

np.random.seed(42)


def density_matrix(state):
    return state * np.conj(state).T


label_0 = [[1],[0],[0],[0]]
label_1 = [[0],[0],[0],[1]]
state_labels = [label_0, label_1]

In [3]:
density_matrix(state_labels[0])

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [4]:
X_tra = []
Y_ans = []
with open('data_banknote_authentication.txt', 'r') as f:
    x = f.readlines()
random.shuffle(x)
for i in range(len(x)):
    x[i] = x[i][:-1]
    x[i] = x[i].split(',')
    X_tra.append(x[i][:-1])
    Y_ans.append(x[i][-1:])
x
ans = []
for i in range(len(Y_ans)):
    if(Y_ans[i]==min(Y_ans)):
        Y_ans[i]=['0']
    ans.append(Y_ans[i][0])
for i in range(len(X_tra)):
    ans[i]=int(ans[i])
    for j in range(len(X_tra[i])):
        X_tra[i][j]=float(X_tra[i][j])

In [5]:
print(scaler.fit(X_tra))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [6]:
X_tra = scaler.transform(X_tra)

In [7]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def qcircuit(params, x=None, y=None):
    for i in range(len(params)):
        qml.Rot(x[0]*(math.pi),x[1]*(math.pi),x[2]*(math.pi), wires=0)
        qml.Rot(x[2]*(math.pi),x[3]*(math.pi),x[4]*(math.pi), wires=1)
        #qml.Rot(x[2]*(math.pi)/2,x[3]*(math.pi)/2,x[4]*(math.pi)/2, wires=2)
        qml.Rot(params[i][0]*(math.pi),params[i][1]*(math.pi),params[i][2]*(math.pi), wires=0)
        qml.Rot(params[i][0]*(math.pi),params[i][1]*(math.pi),params[i][2]*(math.pi), wires=1)
        #qml.Rot(params[i][2]*(math.pi)/2,params[i][3]*(math.pi)/2,params[i][4]*(math.pi)/2, wires=2)
    return qml.expval(qml.Hermitian(y, wires=[0,1]))


def fidelity(state1, state2):
    return np.abs(np.dot(np.conj(state1), state2))


def cost(params, x, y, state_labels=None):
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        f = qcircuit(params, x=x[i], y=dm_labels[y[i]])
        loss = loss + (1 - f) ** 2
    return loss / len(x)

Utility functions for testing and creating batches
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



In [8]:
def test(params, x, y, state_labels=None):
    fidelity_values = []
    output_states = []
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        expectation = qcircuit(params, x=x[i], y=dm_labels[y[i]])
        output_states.append(dev._state)
    predicted = predicted_labels(output_states, state_labels)
    return predicted, output_states


def predicted_labels(states, state_labels=None):
    output_labels = [
        np.argmax([fidelity(s, label) for label in state_labels]) for s in states
    ]
    return np.array(output_labels)


def accuracy_score(y_true, y_pred):
    score = y_true == y_pred
    return score.sum() / len(y_true)


def iterate_minibatches(inputs, targets, batch_size):
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [9]:
Xdata, Xtest, y_train, y_test = train_test_split(X_tra, ans, test_size=0.33, random_state=42)
Xdata = np.array(Xdata)
Xtest = np.array(Xtest)

Train a quantum classifier on the circle dataset
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



In [10]:
# Generate training and test data
num_training = 200
num_test = 2000

#Xdata, y_train = circle(num_training)
X_train = np.hstack((Xdata, np.zeros((Xdata.shape[0], 1))))

#Xtest, y_test = circle(num_test)
X_test = np.hstack((Xtest, np.zeros((Xtest.shape[0], 1))))


# Train using Adam optimizer and evaluate the classifier
num_layers = 3
learning_rate = 0.6
epochs = 10
batch_size = 32

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

# initialize random weights
params = np.random.uniform(size=(num_layers, 3))
print(params)
predicted_train, states_train = test(params, X_train, y_train, state_labels)
accuracy_train = accuracy_score(y_train, predicted_train)

predicted_test, states_test = test(params, X_test, y_test, state_labels)
accuracy_test = accuracy_score(y_test, predicted_test)

# save predictions with random weights for comparison
initial_predictions = predicted_test

loss = cost(params, X_test, y_test, state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test))

for it in range(epochs):
    for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
        params = opt.step(lambda v: cost(v, Xbatch, ybatch, state_labels), params)

    predicted_train, states_train = test(params, X_train, y_train, state_labels)
    accuracy_train = accuracy_score(y_train, predicted_train)
    loss = cost(params, X_train, y_train, state_labels)

    predicted_test, states_test = test(params, X_test, y_test, state_labels)
    accuracy_test = accuracy_score(y_test, predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res))

[[0.37454012 0.95071431 0.73199394]
 [0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501]]
Epoch:  0 | Cost: 0.623473 | Train accuracy: 0.396083 | Test Accuracy: 0.410596
Epoch:  1 | Loss: 0.468810 | Train accuracy: 0.749728 | Test accuracy: 0.726269
Epoch:  2 | Loss: 0.402466 | Train accuracy: 0.754081 | Test accuracy: 0.730684
Epoch:  3 | Loss: 0.423290 | Train accuracy: 0.625680 | Test accuracy: 0.609272
Epoch:  4 | Loss: 0.474231 | Train accuracy: 0.572361 | Test accuracy: 0.562914
Epoch:  5 | Loss: 0.434115 | Train accuracy: 0.623504 | Test accuracy: 0.602649
Epoch:  6 | Loss: 0.419569 | Train accuracy: 0.797606 | Test accuracy: 0.772627
Epoch:  7 | Loss: 0.415515 | Train accuracy: 0.785637 | Test accuracy: 0.801325
Epoch:  8 | Loss: 0.436349 | Train accuracy: 0.669206 | Test accuracy: 0.646799
Epoch:  9 | Loss: 0.633965 | Train accuracy: 0.426551 | Test accuracy: 0.423841
Epoch: 10 | Loss: 0.464288 | Train accuracy: 0.620239 | Test accuracy: 0.578366


Results
~~~~~~~



In [2]:
#print("Cost: {:3f} | Train accuracy {:3f} | Test Accuracy : {:3f}".format(
 #       loss, accuracy_train, accuracy_test))

print("Learned weights")
for i in range(num_layers):
    print("Layer {}: {}".format(i, params[i]))


fig, axes = plt.subplots(1, 3, figsize=(10, 3))
plot_data(X_test, initial_predictions, fig, axes[0])
plot_data(X_test, predicted_test, fig, axes[1])
plot_data(X_test, y_test, fig, axes[2])
axes[0].set_title("Predictions with random weights")
axes[1].set_title("Predictions after training")
axes[2].set_title("True test data")
plt.show()

Learned weights


NameError: name 'num_layers' is not defined

This tutorial was generated using the following Pennylane version:



In [ ]:
qml.about()

References
----------
[1] Pérez-Salinas, Adrián, et al. “Data re-uploading for a universal
quantum classifier.” arXiv preprint arXiv:1907.02085 (2019).

[2] Kingma, Diederik P., and Ba, J. "Adam: A method for stochastic
optimization." arXiv preprint arXiv:1412.6980 (2014).

[3] Liu, Dong C., and Nocedal, J. "On the limited memory BFGS
method for large scale optimization." Mathematical programming
45.1-3 (1989): 503-528.

